### **Quickstart**

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

In [3]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage("Chào , tôi là kytranmoi")])


AIMessage(content='Chào bạn, kytranmoi! Bạn có muốn hỏi hay chia sẻ điều gì không?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 16, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-f6124c69-2ca4-4791-be13-56d6c6a6342b-0', usage_metadata={'input_tokens': 16, 'output_tokens': 20, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
llm.invoke([HumanMessage("Tên tôi là gì ?")])

AIMessage(content='Xin lỗi, nhưng tôi không biết tên của bạn. Bạn có thể cho tôi biết tên của bạn được không?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 12, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-88b4e4e6-f689-4b78-b72a-186c6b0e7d1c-0', usage_metadata={'input_tokens': 12, 'output_tokens': 23, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- Thì như ví dụ ở trên chúng ta thấy nó không hiểu được lịch sử của cuộc trò chuyện.
    -   Điều này thật là tệ hại với một con chatbot trong nó không khác gì so với việc tự đi search google.
- Để giải quyết vấn đề này chúng ta cần truyền vô toàn bộ lịch sử của cuộc trò chuyện.

In [5]:
from langchain_core.messages import AIMessage

llm.invoke(
    [
        HumanMessage("Chào , tôi là kytranmoi"),
        AIMessage("Chào bạn, kytranmoi! Bạn có muốn hỏi hay chia sẻ điều gì không?"),
        HumanMessage("Tôi là ai ?"),
    ]
)

AIMessage(content='Bạn là kytranmoi! Tuy nhiên, tôi không có thông tin cụ thể về bạn. Nếu bạn muốn chia sẻ thêm về bản thân, tôi rất vui được lắng nghe!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 49, 'total_tokens': 87, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_reason': 'stop', 'logprobs': None}, id='run-c0a89fac-95d8-4f7a-a3d9-7212252d73db-0', usage_metadata={'input_tokens': 49, 'output_tokens': 38, 'total_tokens': 87, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- Như vậy nó đã có thể phản hồi dựa trên lịch sử cũ của cuộc trò truyện.
- Đây là ý tưởng cơ bản để hỗ trợ khả nawg=ng tương tác đàm thoại của Chatbot. Vậy làm thế nào để triển khai điều này tốt nhất?

### **Message persistence**

- LangGraph triển khai lớp duy trì tích hợp, khiến nó trở nên lý tưởng cho các ứng dụng trò chuyện hỗ trợ nhiều lượt trò chuyện.
- Việc gói mô hình trò chuyện của chúng tôi trong ứng dụng LangGraph tối giản cho phép chúng tôi tự động lưu lại lịch sử tin nhắn, giúp đơn giản hóa quá trình phát triền các ứng dụng nhiều bước.
- LangGraph đi kèm với một checkpointer trong bộ nhớ đơn giản, chúng tôi sử dụng bên dưới.

In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = llm.invoke(state['messages'])
    return {"messages": response}

workflow.add_edge(START, "llm")
workflow.add_node("llm", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


- Bây giờ chúng ta cần tạo một configmà chúng ta truyền vào runnable mọi lúc. Cấu hình này chứa thông tin không phải là một phần của đầu vào trực tiếp, nhưng vẫn hữu ích. Trong trường hợp này, chúng ta muốn bao gồm một thread_id. Nó sẽ trông như thế này:

In [12]:
config = {"configurable": {"thread_id": "abc123"}}

- Điều này cho phép chúng tôi hỗ trợ nhiều luồng hội thoại với một ứng dụng duy nhất, một yêu cầu phổ biến khi ứng dụng của bạn có nhiều người dùng.

In [13]:
query = """
Chào , tôi là kytranmoi
"""
input_message = [HumanMessage(query)]
output = app.invoke({"messages":input_message}, config=config)
output['messages'][-1].pretty_print()

================================== Ai Message ==================================

Chào bạn, kytranmoi! Rất vui được gặp bạn. Bạn cần giúp gì hôm nay?


In [18]:
query = "Tên tôi là gì ?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tên bạn là kytranmoi. Nếu bạn cần tôi giúp gì khác, hãy cho tôi biết nhé!


- Tuyệt! Chatbot của chúng tôi hiện ghi nhớ mọi thứ về chúng tôi. Nếu chúng tôi thay đổi cấu hình để tham chiếu đến một thread_id, chúng tôi có thể thấy rằng nó bắt đầu cuộc trò chuyện mới.

In [19]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Xin lỗi, nhưng tôi không biết tên của bạn. Bạn có muốn chia sẻ tên của mình không?


In [20]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tên bạn là kytranmoi. Nếu bạn có câu hỏi nào khác hoặc cần trợ giúp, hãy cho tôi biết!


- Đây là cách chúng ta có thể hỗ trợ chatbot trò chuyện với nhiều người dùng!

**Mẹo**
- Để hỗ trợ bất đồng bộ, hãy cập nhật call_modelnút thành hàm bất đồng bộ và sử dụng .ainvoke khi gọi ứng dụng: